In [1]:
import os
import gc
import sys
import pickle
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import time
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from IPython.display import clear_output
from matplotlib import pyplot as plt 
from tqdm import tqdm

In [2]:
!pip install space-bandits
from space_bandits import BanditAlgorithm
clear_output()

In [ ]:
## plug in gdrive to load the data from gdrive
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
## path to the contextual_dataset_wu.py file
path = '/content/drive/MyDrive/Fellowship_Deep_and_Wide_Bandit/'
sys.path.append(path)

from contextual_dataset_wu import ContextualDataset

In [4]:
## Load a subset of the dataset

p = 0.01  # Read 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
data_snippet = pd.read_csv(
         path+'data_snippet.csv',
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p ## skip rows unless randomly generated number < p
)

In [ ]:
## Use label encoding for campaign_type
data_snippet["campaign_type"] = data_snippet["campaign_type"].astype('category')
data_snippet["campaign_type_cat"] = data_snippet["campaign_type"].cat.codes

## Set rewards
data_snippet['rewards'] = data_snippet['opened'] 
data_snippet['rewards'][data_snippet['unsub']==1] = -2
data_snippet['rewards'][data_snippet['opened']==0] = -1
data_snippet['rewards'] = data_snippet['rewards'] + data_snippet['rev_3dv2'] / 100.

clear_output()

In [6]:
## Define Dataset for data snippet
class BanditTestDataset(Dataset):
    def __init__(self, data, user_col, context_col, rewards_col, action_col=None):
        super(BanditTestDataset, self).__init__()
        ## data - pandas dataframe
        ## user_col - user IDs column name
        ## action_col - action column name
        ## context_col - context column names
        ## rewards_col - reward column name
        self.user_ids = data[user_col]
        self.context = data[context_col]
        self.rewards = data[rewards_col]
        if action_col != None:
          self.actions = data[action_col]
        
    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        ## returns numpy arrays
        user_id = self.user_ids.iloc[index].values[0]
        context = self.context.iloc[index].values
        reward = self.rewards.iloc[index].values[0]
        reward_orig = reward ## copy the reward before changing it
        if action_col != None:
          action = self.actions.iloc[index].values[0]
        else:
          ## Randomly choose to send or not send email
          randnum = random.random() ## draw a random number between 0 and 1
          threshold = 0.5 ## e.g., if thresehold = 0.2, send email 80% of the time
          if randnum >= threshold:
            action = 1 ## send email if random number larger than threshold, get the associated reward
          else:
            action = 0 ## don't send email if random number smaller than threshold
            reward = reward * -1. ## Get opposite reward compared to send email

        return user_id, action, context, reward, reward_orig

In [7]:
## Split 80/20 into train and val. Create dataloaders.
## val not used in this version, just trying to run space-bandits at this point...

train, val = train_test_split(data_snippet, test_size=0.2)

user_col = ['riid']
context_col = ['retention_score','recency_score','frequency_score','campaign_type_cat']
rewards_col = ['rewards']
action_col = None

batch_size = 2048

train_dataset = BanditTestDataset(train, user_col, context_col, rewards_col, action_col=None)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
del train

val_dataset = BanditTestDataset(val, user_col, context_col, rewards_col, action_col=None)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
del val

In [8]:
num_actions = 2 ## Send email or not send
num_features = len(context_col)
num_users = data_snippet[user_col].nunique()[0]
print("Number of actions:", num_actions)
print("Number of features:", num_features)
print("Number of users:", num_users)

Number of actions: 2
Number of features: 4
Number of users: 57792


In [9]:
class Wide_Model(nn.Module):
    def __init__(self, embed_size=100, n_action=2, embed_dim=64):
        ## Learns expected reward for each action given User ID
        ## Uses embeddings to 'memorize' individual users
        ## embed_size - size of the dictionary of embeddings
        ## embed_dim -  size of each embedding vector
        ## n_action - number of possible actions
        super(Wide_Model, self).__init__()
        self.embed_size = embed_size
        self.n_action = n_action
        self.embed_dim = embed_dim
        
        self.embedding = nn.Embedding(self.embed_size, self.embed_dim)
        self.lr = nn.Linear(self.embed_dim, self.n_action)
    
    def forward(self, x):
        ## Input: user ID
        x = self.embedding(x)
        x = self.lr(x)
        return x.squeeze(axis=0)
    

class Deep_Model(nn.Module):
    def __init__(self, context_size=5, layer_sizes=[50], n_action=2):
        ## Learns expected reward for each action given context
        ## layer_sizes (list of integers): defines neural network architecture: n_layers = len(layer_sizes), 
        ## value is per-layer width. (default [50])
        super(Deep_Model, self).__init__()
        self.context_size = context_size
        self.layer_sizes = layer_sizes
        self.n_action = n_action

        self.layers = []
        self.build_model()
        self.activation = nn.ReLU()
        ##self.activation = nn.Sigmoid()
    
    def build_layer(self, inp_dim, out_dim):
        """Builds a layer in deep model """

        layer = nn.modules.linear.Linear(inp_dim,out_dim)
        nn.init.uniform_(layer.weight)
        name = f'layer {len(self.layers)}'
        self.add_module(name, layer)
        return layer
  
    def build_norm_layer(self, dim):
        """Builds a layer in deep model """

        layer = nn.BatchNorm1d(dim)
        name = f'layer {len(self.layers)}'
        self.add_module(name, layer)
        return layer
    
    def build_model(self):
        """
        Defines the actual NN model with fully connected layers.
        """
        for i, layer in enumerate(self.layer_sizes):
            if i==0:
                inp_dim = self.context_size
            else:
                inp_dim = self.layer_sizes[i-1]
            out_dim = self.layer_sizes[i]
            new_layer = self.build_layer(inp_dim, out_dim)
            self.layers.append(new_layer)

        #norm_layer = self.build_norm_layer(out_dim)
        #self.layers.append(norm_layer)

        output_layer = self.build_layer(out_dim, self.n_action)
        self.layers.append(output_layer)

    def forward(self, x):
        """forward pass of the neural network"""
        ## Input: context
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i != len(self.layers)-1:
                x = self.activation(x)
        return x.squeeze(axis=0)
    

class Wide_and_Deep_Model(nn.Module):
    def __init__(self, context_size=5, deep_layer_sizes=[50,100], n_action=2, embed_size=100, wide_embed_dim=64):
        super(Wide_and_Deep_Model, self).__init__()
        ## Combines the Wide model and Deep model
        self.n_action = n_action
        self.context_size = context_size
        self.deep_layer_sizes = deep_layer_sizes
        self.embed_size = embed_size
        self.wide_embed_dim = wide_embed_dim

        self.wide_model = Wide_Model(embed_size=self.embed_size, n_action=self.n_action, embed_dim=self.wide_embed_dim)
        self.deep_model = Deep_Model(context_size=self.context_size, layer_sizes=self.deep_layer_sizes, n_action=self.n_action)
    
    def forward(self, wide_input, deep_input):
        x_wide = self.wide_model(wide_input)
        x_deep = self.deep_model(deep_input)
        ## Add the output from wide and deep models
        x = x_wide + x_deep

        return x.squeeze(-1), x_wide.squeeze(-1), x_deep.squeeze(-1)

In [10]:
class Test_Wide_Deep_Bandits(BanditAlgorithm):
    def __init__(
        self,
        num_actions,
        num_features,
        wide_embed_size=100, ## Size of embedding dictionary for the wide model
        wide_embed_dim=64, ## Dimension of embedding for the wide model
        update_freq_nn = 1, ## Frequency to update the model, default updates model for every data point
        num_epochs = 1, ## Number of steps to Train for each update
        model_type = 'wide_deep', ## model_type = 'wide', 'deep', or 'wide_deep'
        name='test_deep_bandits'):
      
        hparams = {
                    'num_actions':num_actions,
                    'context_dim':num_features,
                    'max_grad_norm':5.0,
        }

        ## Raise error if model_type is not one of the available models
        possible_models = ['deep','wide','wide_deep']
        if model_type not in possible_models:
          raise NameError('model_type must be "deep", "wide", or "wide_deep"')

        self.name = name
        self.model_type = model_type
        self.wide_embed_dim = wide_embed_dim
        self.wide_embed_size = wide_embed_size
        self.hparams = hparams

        ## Initialize model and optimizer depending on model_type
        if self.model_type == 'deep':
          self.deep_model = Deep_Model(context_size=self.hparams['context_dim'],
                                       n_action=self.hparams['num_actions'])
          self.optim = torch.optim.RMSprop(self.deep_model.parameters())

        if self.model_type == 'wide':
          self.wide_model = Wide_Model(embed_size=self.wide_embed_size, 
                                      n_action=self.hparams['num_actions'], 
                                      embed_dim=self.wide_embed_dim)
          self.optim = torch.optim.RMSprop(self.wide_model.parameters())
        

        if self.model_type == 'wide_deep':
          self.wide_deep_model = Wide_and_Deep_Model(context_size=self.hparams['context_dim'],
                                                    embed_size=self.wide_embed_size, 
                                                    n_action=self.hparams['num_actions'], 
                                                    wide_embed_dim=self.wide_embed_dim) 
          self.optim = torch.optim.RMSprop(self.wide_deep_model.parameters())
        
        self.loss = nn.modules.loss.MSELoss()

        self.t = 0
        self.update_freq_nn = update_freq_nn 
        self.num_epochs = num_epochs  
        self.data_h = ContextualDataset(self.hparams['context_dim'],
                                        self.hparams['num_actions'],
                                        intercept=False)
        
        ## Keep a dictionary of users that matches user's riid to indexes between 0 and num_users
        ## Initialize dicitonary with a "dummy user" that will be used for prediction when the user has never been seen
        self.user_dict = {0:0} 
        self.current_user_size = 1

    def expected_values(self, user_id, context):

        context = torch.tensor(context).float()
        user_id = self.lookup_one_user(user_id)

        if self.model_type == 'deep':
          x = self.deep_model(context)
        if self.model_type == 'wide':
          x = self.wide_model(user_id)
        if self.model_type == 'wide_deep':
          x, x_wide, x_deep = self.wide_deep_model(user_id, context)
          if user_id == 0:
            x = x_deep ## Return only the deep model output if user is a "dummy user" 
        #return x, x_wide, x_deep
        return x

    def action(self, user_id, context):
        ## Select and action based on expected values of reward
        if self.model_type == 'deep':
          vals = self.expected_values(user_id, context)  ## for deep model
        if self.model_type == 'wide':
          vals = self.expected_values(user_id, context)   ## for wide model
        if self.model_type == 'wide_deep':
          vals = self.expected_values(user_id, context)  ## for wide and deep model
        return np.argmax(vals.detach().numpy())
        
    def update(self, user_id, context, action, reward):
        """
        Args:
          user_id: Last User ID
          context: Last observed context.
          action: Last observed action.
          reward: Last observed reward.
        """
        
        self.t += 1
        self.data_h.add(user_id, context, action, reward)
        self.update_user_dict(user_id)

        if self.t % self.update_freq_nn == 0:
          self.train(self.data_h, self.num_epochs)
        
        
    def do_step(self, u, x, y, w, step):
        if self.model_type == 'deep':
          y_hat = self.deep_model.forward(x.float())
        if self.model_type == 'wide':
          y_hat = self.wide_model.forward(u)
        if self.model_type == 'wide_deep':
          y_hat, y_wide, y_deep = self.wide_deep_model(u,x.float())
        
        y_hat *= w
        ls = self.loss(y_hat, y.float())
        ls.backward()

        clip = self.hparams['max_grad_norm']

        if self.model_type == 'deep':
          torch.nn.utils.clip_grad_norm_(self.deep_model.parameters(), clip)
        if self.model_type == 'wide':
          torch.nn.utils.clip_grad_norm_(self.wide_model.parameters(), clip)
        if self.model_type == 'wide_deep':
          torch.nn.utils.clip_grad_norm_(self.wide_deep_model.parameters(), clip)

        self.optim.step()
        self.optim.zero_grad()

    def train(self, data, num_steps):
        """Trains the network for num_steps, using the provided data.
        Args:
          data: ContextualDataset object that provides the data.
          num_steps: Number of minibatches to train the network for.
        Takes longer to get batch data and train model as the data size increase
        """
        #print("Training at time {} for {} steps...".format(self.t, num_steps))

        batch_size = 512
        
        data.scale_contexts() ## have to scale the data first if scaled=True in data.get_batch_with_weights()

        for step in range(num_steps):
            u, x, y, w = data.get_batch_with_weights(batch_size, scaled=True)
            u = self.lookup_users(u)

            ## Training at time step 1 will cause problem if scaled=True, 
            ## because standard deviation=0, and scaled_context will equal nan
            if self.t != 1:
              self.do_step(u, x, y, w, step)
        

    def lookup_one_user(self, user_id):
      user_id = user_id.tolist()
      if user_id not in self.user_dict.keys():
        user_index = 0
      else:
        user_index = self.user_dict[user_id]
      return torch.tensor(user_index)

    def lookup_users(self, user_ids):
      ## Returns a list of user indexes for input to the wide network
      user_ids = user_ids.tolist()
      user_index = [self.user_dict[u] for u in user_ids]
      return torch.tensor(user_index)
    
    def update_user_dict(self, user_id):
      ## Create/update a lookup dictionary that matches user ID to a user index between 0 and num_users
      user_id = user_id.tolist()
      if user_id not in self.user_dict:
        self.user_dict.update({user_id:self.current_user_size})
        self.current_user_size += 1
              
    def save(self, path):
        """saves model to path"""
        with open(path, 'wb') as f:
            pickle.dump(self, f)

In [11]:
model_wide = Test_Wide_Deep_Bandits(num_actions, num_features, wide_embed_size=num_users, model_type='wide', update_freq_nn = 100, num_epochs = 1)
model_deep = Test_Wide_Deep_Bandits(num_actions, num_features, wide_embed_size=num_users, model_type='deep', update_freq_nn = 100, num_epochs = 1)
model_wide_deep = Test_Wide_Deep_Bandits(num_actions, num_features, wide_embed_size=num_users, model_type='wide_deep', update_freq_nn = 100, num_epochs = 1)

In [12]:
%%time
tbar = tqdm(train_dataloader)
for item in tbar:
  for i in range(batch_size):
    try:
      user1 = item[0][i]
      action1 = item[1][i]
      context1 = item[2][i]
      reward1 = item[3][i]
    except:
      break
    
    model_wide.update(torch.tensor(user1),context1,action1,reward1)
    model_deep.update(torch.tensor(user1),context1,action1,reward1)
    model_wide_deep.update(torch.tensor(user1),context1,action1,reward1)

#clear_output()

  0%|          | 0/25 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 25/25 [1:05:22<00:00, 156.91s/it]

CPU times: user 1h 4min 27s, sys: 46.8 s, total: 1h 5min 14s
Wall time: 1h 5min 22s


In [21]:
## Save models
model_wide.save(path+'ds_model_wide.pkl')
model_deep.save(path+'ds_model_deep.pkl')
model_wide_deep.save(path+'ds_model_wide_deep.pkl')

In [ ]:
## Load existing models
#model_wide = load_model(path+'ds_model_wide.pkl')
#model_deep = load_model(path+'ds_model_deep.pkl')
#model_wide_deep = load_model(path+'ds_model_wide_deep.pkl')

In [17]:
tbar = tqdm(val_dataloader)

y_true = []
y_pred_deep = []
y_pred_wide = []
y_pred_wide_deep = []

num_correct_deep = 0
num_correct_wide = 0
num_correct_wide_deep = 0
total = 0

## Context means and std for scaling 
context_means = data_snippet[context_col].mean().values
context_std = data_snippet[context_col].std().values

for item in tbar:
  for i in range(batch_size):
    try:
      user1 = item[0][i]
      action1 = item[1][i]
      context1 = item[2][i]
      context1_scaled = (context1 - context_means) / context_std ## Scale context for input 
      reward_1 = item[3][i]
      reward_o = item[4][i]
    except:
      break

    if reward_o >= 0.0:
      best_action = 1
    else:
      best_action = 0
      
    y_true.append(best_action)

    ## Predict best action: REMEMBER TO SCALE CONTEXT!!!!
    model_action_deep = model_deep.action(torch.tensor(user1),context1_scaled) 
    model_action_wide = model_wide.action(torch.tensor(user1),context1_scaled) 
    model_action_wide_deep = model_wide_deep.action(torch.tensor(user1),context1_scaled) 

    if best_action == model_action_deep:
      num_correct_deep += 1
    if best_action == model_action_wide:
      num_correct_wide += 1
    if best_action == model_action_wide_deep:
      num_correct_wide_deep += 1
    
    total += 1

    y_pred_deep.append(model_action_deep)
    y_pred_wide.append(model_action_wide)
    y_pred_wide_deep.append(model_action_wide_deep)

clear_output()

print("Accuracy of wide model = ", num_correct_wide / total)
print("Accuracy of deep model = ", num_correct_deep / total)
print("Accuracy of wide and deep model = ", num_correct_wide_deep / total)
    

Accuracy of wide model =  0.5916606757728253
Accuracy of deep model =  0.7808131639907341
Accuracy of wide and deep model =  0.7717868839364167


In [18]:
confusion_matrix(y_true,y_pred_deep)

array([[3828, 1289],
       [1455, 5947]])

In [19]:
confusion_matrix(y_true,y_pred_wide)

array([[ 132, 4985],
       [ 127, 7275]])

In [20]:
confusion_matrix(y_true,y_pred_wide_deep)

array([[4019, 1098],
       [1759, 5643]])